# steps

1. run get_img_to_yolo_ouput.py.ipynb, get true values (image_to_cells_true.jl)
2. run detect_server_1007/ev.py, get pred values (image_to_cells_pred.jl)
3. iou
4. acc

# 1. test

In [155]:
import joblib as jl

image_to_cells_true = jl.load('image_to_cells_true.jl')
image_to_cells_pred = jl.load('image_to_cells_pred.jl')

In [156]:
type(image_to_cells_true),len(image_to_cells_true),type(image_to_cells_pred),len(image_to_cells_pred)

(dict, 1328, dict, 10)

In [166]:
image_to_cells_pred['2559636b-f01a-4414-93da-210c3b12d153.png'][:10],len(image_to_cells_pred['2559636b-f01a-4414-93da-210c3b12d153.png'])

([(127, 405, 249, 524),
  (342, 69, 456, 194),
  (1039, 693, 1167, 820),
  (1192, 50, 1318, 179),
  (1075, 295, 1192, 415),
  (217, 802, 337, 938),
  (926, 650, 1041, 765),
  (298, 191, 408, 302),
  (1226, 738, 1352, 862),
  (397, 958, 537, 1091)],
 177)

In [167]:
image_to_cells_pred['2559636b-f01a-4414-93da-210c3b12d153.png'][175:]

[(581, 173, 716, 317), (839, 141, 1012, 297)]

In [158]:
image_to_cells_true['2559636b-f01a-4414-93da-210c3b12d153.png'][:10],len(image_to_cells_true['2559636b-f01a-4414-93da-210c3b12d153.png'])

([(688.0, 618.0, 810.0, 744.0),
  (1035.0, 689.0, 1168.0, 819.0),
  (422.0, 661.0, 558.0, 803.0),
  (1205.0, 417.0, 1313.0, 524.0),
  (412.0, 424.0, 513.0, 525.0),
  (534.0, 524.0, 660.0, 633.0),
  (298.0, 189.0, 407.0, 302.0),
  (342.0, 75.0, 455.0, 196.0),
  (1075.0, 290.0, 1191.0, 416.0),
  (859.0, 993.0, 991.0, 1094.0)],
 49)

# 2. calculation iou

In [159]:
def get_iou(c1, c2):
    x1_min,y1_min,x1_max,y1_max = c1
    x2_min,y2_min,x2_max,y2_max = c2
    left_column_max = max(x1_min,x2_min)
    right_column_min = min(x1_max,x2_max)
    up_row_max = max(y1_min,y2_min)
    down_row_min = min(y1_max,y2_max)
    if left_column_max>=right_column_min or up_row_max>=down_row_min:
        return 0
        
    area_cross = (right_column_min-left_column_max)*(down_row_min-up_row_max)
    area_s1 = (x1_max-x1_min)*(y1_max-y1_min)
    area_s2= (x2_max-x2_min)*(y2_max-y2_min)
    return area_cross/(area_s1+area_s2-area_cross)

# 3. iteration each pred

In [160]:
import joblib as jl

def get_correct_pred_cell_count(image_name):
    image_to_cells_true = jl.load('image_to_cells_true.jl')
    image_to_cells_pred = jl.load('image_to_cells_pred.jl')
    cells_pred = image_to_cells_pred[image_name]
    cells_true = image_to_cells_true[image_name]
    
    dup_true_cells = []
    correct_pred_count = 0
    for cell_pred in cells_pred:
        for cell_true in cells_true:
            iou = get_iou(cell_pred, cell_true)
            if iou>0.8 and cell_true not in dup_true_cells:
                correct_pred_count+=1
                dup_true_cells.append(cell_true)
                break
    return correct_pred_count

In [161]:
get_correct_pred_cell_count('2559636b-f01a-4414-93da-210c3b12d153.png')

49

In [165]:
image_to_cells_pred = jl.load('image_to_cells_pred.jl')
all_cell_num_pred = 0
all_cell_num_pred_correct = 0
for image_name in image_to_cells_pred:
    print('processing {0}'.format(image_name))
    cell_num_pred = len(image_to_cells_pred[image_name])
    cell_num_pred_correct = get_correct_pred_cell_count(image_name)
    all_cell_num_pred+=cell_num_pred
    print('all_cell_num_pred={0},cell_num_pred={1}'.format(all_cell_num_pred,cell_num_pred))
    all_cell_num_pred_correct+=cell_num_pred_correct
acc = all_cell_num_pred_correct/all_cell_num_pred

processing 065af549-320a-4862-b92a-8ef750d7b5fb.png
all_cell_num_pred=276,cell_num_pred=276
processing 0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png
all_cell_num_pred=588,cell_num_pred=312
processing 10be6380-cbbb-4886-8b9e-ff56b1710576.png
all_cell_num_pred=654,cell_num_pred=66
processing 13099edb-35d9-438f-b093-2cf2ebf9d255.png
all_cell_num_pred=804,cell_num_pred=150
processing 2559636b-f01a-4414-93da-210c3b12d153.png
all_cell_num_pred=981,cell_num_pred=177
processing 67acd5bc-4658-4c6e-a9b9-cd8ea3a03418.png
all_cell_num_pred=1269,cell_num_pred=288
processing 6b14c855-8561-417c-97a4-63fa552842fd.png
all_cell_num_pred=1341,cell_num_pred=72
processing 7c66d2fd-68ee-4628-b860-9d7ad25b2a80.png
all_cell_num_pred=1577,cell_num_pred=236
processing 84c88143-940d-4b62-bbd2-7acc4cadcb6b.png
all_cell_num_pred=1784,cell_num_pred=207
processing 8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png
all_cell_num_pred=2132,cell_num_pred=348


In [ ]:
print(acc)

In [164]:
all_cell_num_pred

2132

# 4. issue check

In [168]:
image_to_cells_pred

{'065af549-320a-4862-b92a-8ef750d7b5fb.png': [(1337, 141, 1465, 280),
  (1472, 485, 1593, 614),
  (250, 622, 378, 733),
  (842, 265, 972, 386),
  (384, 90, 501, 218),
  (509, 282, 625, 397),
  (884, 932, 1001, 1061),
  (639, 532, 765, 652),
  (1033, 727, 1176, 851),
  (0, 902, 106, 1025),
  (1440, 845, 1568, 969),
  (1157, 986, 1276, 1104),
  (727, 206, 855, 326),
  (260, 873, 381, 984),
  (1245, 905, 1365, 1025),
  (403, 445, 526, 564),
  (746, 55, 861, 176),
  (1452, 681, 1588, 811),
  (569, 633, 691, 748),
  (269, 985, 388, 1093),
  (11, 689, 128, 799),
  (458, 800, 579, 918),
  (1, 1057, 88, 1179),
  (1105, 284, 1234, 404),
  (1045, 950, 1164, 1085),
  (400, 229, 512, 352),
  (444, 987, 582, 1103),
  (711, 617, 834, 728),
  (858, 97, 976, 209),
  (531, 475, 645, 584),
  (598, 388, 701, 503),
  (189, 95, 300, 217),
  (714, 414, 830, 525),
  (677, 1027, 806, 1138),
  (404, 694, 522, 800),
  (51, 224, 161, 347),
  (1014, 456, 1154, 571),
  (633, 751, 749, 856),
  (1104, 546, 1231, 665

In [178]:
cnt=0
for xx in image_to_cells_pred['2559636b-f01a-4414-93da-210c3b12d153.png']:
    x_min,x_max,y_min,y_max = xx
    if (x_max-x_min)*(y_max-y_min)>60000:
        cnt+=1
print(cnt)        

126


In [175]:
len( image_to_cells_pred['2559636b-f01a-4414-93da-210c3b12d153.png'] )

177